In [4]:
!git clone https://github.com/bakhbyergyen7/ComputerVisionProject.git
# We might need to use this to load data since this is an image dataset

fatal: destination path 'ComputerVisionProject' already exists and is not an empty directory.


In [15]:
!pip install torchvision
!pip install timm

In [16]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import timm
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

In [19]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset loading
def load_dataset(base_dir):
    images = []
    labels = []
    for label_dir in os.listdir(base_dir):
        emotion_dir = os.path.join(base_dir, label_dir)
        if os.path.isdir(emotion_dir):
            for img_file in os.listdir(emotion_dir):
                img_path = os.path.join(emotion_dir, img_file)
                images.append(img_path)
                labels.append(label_dir)
    return images, labels

# Load and split dataset
train_images, val_images, train_labels, val_labels = train_test_split(*load_dataset('/content/ComputerVisionProject/project_1_dataset/train'), test_size=0.2, random_state=42)
test_images, test_labels = load_dataset('/content/ComputerVisionProject/project_1_dataset/test')



In [21]:
label_encoder = LabelEncoder()
train_encoded_labels = label_encoder.fit_transform(train_labels)
val_encoded_labels = label_encoder.transform(val_labels)
test_encoded_labels = label_encoder.transform(test_labels)

In [22]:
class FacialExpressionDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        return image, label

In [23]:
train_dataset = FacialExpressionDataset(train_images, train_encoded_labels, transform)
val_dataset = FacialExpressionDataset(val_images, val_encoded_labels, transform)
test_dataset = FacialExpressionDataset(test_images, test_encoded_labels, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load teacher and student models
teacher_model = timm.create_model('deit_base_patch16_224', pretrained=True, num_classes=7)
teacher_model.to(device)
student_model = timm.create_model('deit_base_patch16_224', pretrained=True, num_classes=7)
student_model.to(device)

class DistillationLoss(nn.Module):
    def __init__(self, alpha=0.5, temperature=2.0):
        super().__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.KLDivLoss = nn.KLDivLoss(reduction='batchmean')
        self.CELoss = nn.CrossEntropyLoss()

    def forward(self, student_output, labels, teacher_output):
        soft_loss = self.KLDivLoss(
            torch.log_softmax(student_output/self.temperature, dim=1),
            torch.softmax(teacher_output/self.temperature, dim=1)
        ) * (self.alpha * self.temperature * self.temperature)

        hard_loss = self.CELoss(student_output, labels) * (1 - self.alpha)
        return soft_loss + hard_loss

criterion = DistillationLoss()
optimizer = optim.Adam(student_model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [25]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model

# Evaluate model function
def evaluate_model(model, loader, criterion, teacher_model=None):
    model.eval()
    if teacher_model:
        teacher_model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if teacher_model:
                teacher_outputs = teacher_model(images)
                loss = criterion(outputs, labels, teacher_outputs)
            else:
                # Fallback if no teacher model provided
                loss = criterion(outputs, labels, outputs)

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    conf_matrix = confusion_matrix(all_labels, all_preds)
    return epoch_loss, epoch_acc, conf_matrix

In [27]:
def train_model(teacher_model, student_model, criterion, optimizer, scheduler, train_loader, val_loader, epochs=250, patience=20):
    best_val_loss = float('inf')
    no_improve_epochs = 0
    train_acc_history = []
    val_acc_history = []
    train_loss_history = []
    val_loss_history = []

    for epoch in range(epochs):
        student_model.train()
        train_loss = 0.0
        train_corrects = 0
        train_total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            # Forward pass through the teacher model
            with torch.no_grad():
                teacher_outputs = teacher_model(images)

            # Forward pass through the student model
            student_outputs = student_model(images)
            loss = criterion(student_outputs, labels, teacher_outputs)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(student_outputs, 1)
            train_loss += loss.item() * images.size(0)
            train_corrects += torch.sum(preds == labels.data)
            train_total += labels.size(0)

        train_epoch_loss = train_loss / train_total
        train_epoch_acc = train_corrects.double() / train_total
        train_acc_history.append(train_epoch_acc.item())
        train_loss_history.append(train_epoch_loss)

        # Validation phase
        val_epoch_loss, val_epoch_acc, _ = evaluate_model(student_model, val_loader, criterion, teacher_model)
        val_acc_history.append(val_epoch_acc.item())
        val_loss_history.append(val_epoch_loss)

        print(f'Epoch {epoch+1}/{epochs} - Train Loss: {train_epoch_loss:.4f}, Acc: {train_epoch_acc:.4f}')
        print(f'Validation Loss: {val_epoch_loss:.4f}, Acc: {val_epoch_acc:.4f}')

        if val_epoch_loss < best_val_loss:
            best_val_loss = val_epoch_loss
            no_improve_epochs = 0
            save_model(student_model, '/home/yuying/841/deit_distilled_model.pth')
            print('Model saved')
        else:
            no_improve_epochs += 1
            if no_improve_epochs >= patience:
                print('Early stopping triggered due to no improvement.')
                break

        scheduler.step()

    # Plotting accuracy and loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(train_acc_history)+1), train_acc_history, label='Train Accuracy')
    plt.plot(range(1, len(val_acc_history)+1), val_acc_history, label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(train_loss_history)+1), train_loss_history, label='Train Loss')
    plt.plot(range(1, len(val_loss_history)+1), val_loss_history, label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('/home/yuying/841/final_deit_results.png')
    plt.close()

In [28]:
def test_model(model, loader, criterion):
    test_loss, test_accuracy, test_conf_matrix = evaluate_model(model, loader, criterion)
    print(f'Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print(f'Test Confusion Matrix:\n{test_conf_matrix}')
    return test_accuracy, test_conf_matrix


In [32]:
mode = 'test'  # Options: 'train', 'test', 'both'

if mode == 'train' or mode == 'both':
    trained_student_model = train_model(teacher_model, student_model, criterion, optimizer, scheduler, train_loader, val_loader)
    print("Training completed.")

if mode == 'test' or mode == 'both':
    student_model = load_model(student_model, '/content/deit_distilled_model.pth')
    test_accuracy, test_conf_matrix = test_model(student_model, test_loader, criterion)
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print(f'Test Confusion Matrix:\n{test_conf_matrix}')



Test Loss: 0.6578
Test Accuracy: 0.7016
Test Confusion Matrix:
[[ 411    1   51   17   81  146   20]
 [  17   40    1    2    4   14    0]
 [  60    2  323   19   63  174   80]
 [  15    0    6 1200   63   43   21]
 [  37    0   32   35  677  174    7]
 [  56    2   52   11  120  494    6]
 [   6    1   37   15   11   36  472]]
Test Accuracy: 0.7016
Test Confusion Matrix:
[[ 411    1   51   17   81  146   20]
 [  17   40    1    2    4   14    0]
 [  60    2  323   19   63  174   80]
 [  15    0    6 1200   63   43   21]
 [  37    0   32   35  677  174    7]
 [  56    2   52   11  120  494    6]
 [   6    1   37   15   11   36  472]]
